<a href="https://colab.research.google.com/github/anunuclse/Final_tesk_DE_step12/blob/main/Final_tesk_DE_step12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>